# Entity extraction using Fasttext and LSTM

## Import everything important

In [2]:
import joblib
import torch
import torch.nn as nn
import transformers
import nltk

import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn import model_selection

# from gensim.models import FastText as ft
from gensim.models import fasttext as ft
from torch.utils.tensorboard import SummaryWriter
from nltk.tokenize import word_tokenize

from tqdm import tqdm

## Some config

In [3]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 128 #
VALID_BATCH_SIZE = 128 #
EPOCHS = 5
EMBED_DIM = 300

MODEL_PATH = "./state_dict.pt"
TRAINING_FILE = 'ner_dataset.csv' #

In [4]:
device = (
    "cuda:0"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
) #
device

'cuda:0'

https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz

In [5]:
# fasttext_model = ft.load_fasttext_format("cc.en.300.bin")
fasttext_model = ft.load_facebook_vectors("cc.en.300.bin")

## Dataset

In [6]:
class EntityDataset:
    def __init__(self, texts, tags):
        self.texts = texts
        self.tags = tags
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        text = self.texts[item]
        tags = np.array(self.tags[item])
    
        # ids = [fasttext_model.wv[s] for s in text]
        ids = [fasttext_model[s] for s in text]
        
# реализуем паддинг: если текст меньше MAX_LEN, 
# забиваем матрицу нулями так, чтобы матрицы были одинакового размера для каждого текста   
# возвращаем матрицу с нулями, список тегов с нулями, и маску -- разметку, какие элементы 
# являются словами, а какие -- пустые поля (нули). Маска нужна, чтобы правильно считать лосс
# то есть, не учитывать в нем "пустые" части с нулями
        
        if len(ids) < MAX_LEN:
            ids_pad = np.array(ids + [[0]*len(ids[0])]*(MAX_LEN - len(ids)))
            tags_pad = list(tags) + [0]*(MAX_LEN - len(tags))
            mask = [1]*len(ids) + [0]*(MAX_LEN - len(ids))
        else:
            ids_pad = np.array(ids[:MAX_LEN])
            tags_pad = list(tags)[:MAX_LEN]
            mask = [1]*MAX_LEN
            
        return (torch.tensor(ids_pad, dtype=torch.float32),
                torch.tensor(tags_pad, dtype=torch.long),
                torch.tensor(mask, dtype=torch.long))

## Training and evaluation functions

In [7]:
def loss_fn(output, target, mask, num_labels):
    lfn = nn.CrossEntropyLoss()
    active_loss = mask.view(-1) == 1
    active_logits = output.view(-1, num_labels)
    active_labels = torch.where(
        active_loss,
        target.view(-1),
        torch.tensor(lfn.ignore_index).type_as(target)
    )
    loss = lfn(active_logits, active_labels)
    return loss

In [8]:
def acc_stat(pred, target, mask):
    mask = mask.bool()
    pred = torch.masked_select(pred, mask)
    target = torch.masked_select(target, mask)
    correct = torch.tensor(torch.eq(pred, target).sum().item(),dtype=torch.float32) # сколько элементов угадано корректно
    total = torch.tensor(len(pred), dtype=torch.float32) # сколько элементов было всего, не считая "пустых" с нулями
    return correct, total

пример того как должно работать

In [9]:
acc_stat(torch.tensor([1,2,3,4,0,0,0,0]), torch.tensor([1,2,3,4,5,5,5,5]), torch.tensor([1,1,1,1,0,0,0,0]))

(tensor(4.), tensor(4.))

In [10]:
acc_stat(torch.tensor([1,2,3,4,0,0,0,0]), torch.tensor([1,2,3,4,5,5,5,5]), torch.tensor([0,0,0,0,1,1,1,1]))

(tensor(0.), tensor(4.))

## Loss function and model

In [11]:
class EntityModel(nn.Module):
    def __init__(self, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5, bidirectional=False):
        super().__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob,
                            batch_first=True, bidirectional=bidirectional)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(
        self, 
        embeds,
        hidden
    ):
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
# здесь пропустим через дропаут и линейный слой
        dropout_out = self.dropout(lstm_out)
        out = self.fc(dropout_out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        num_directions = 2 if self.lstm.bidirectional else 1
        print("num_directions", num_directions)
        h_zeros = torch.zeros(self.n_layers * num_directions,
                              batch_size, self.hidden_dim,
                              dtype=torch.float32, device=device)
        c_zeros = torch.zeros(self.n_layers * num_directions,
                              batch_size, self.hidden_dim,
                              dtype=torch.float32, device=device)

        return (h_zeros, c_zeros)

## Data processing

In [12]:
def process_data(data_path):
    df = pd.read_csv(data_path, encoding="latin-1")
    df.loc[:, "Sentence #"] = df["Sentence #"].fillna(method="ffill")

    enc_tag = preprocessing.LabelEncoder()

    # df.loc[:, "Tag"] = enc_tag.fit_transform(df["Tag"])
    df[df.columns[df.columns.get_loc("Tag")]] = enc_tag.fit_transform(df["Tag"])

    sentences = df.groupby("Sentence #")["Word"].apply(list).values
    tag = df.groupby("Sentence #")["Tag"].apply(list).values
    return sentences, tag, enc_tag

## Training

In [13]:
sentences, tag, enc_tag = process_data(TRAINING_FILE)

In [14]:
from sklearn.model_selection import train_test_split #

meta_data = {
    "enc_tag": enc_tag
}

joblib.dump(meta_data, "meta.bin")

num_tag = len(list(enc_tag.classes_))

(
    train_sentences,
    test_sentences,
    train_tag,
    test_tag
) = train_test_split(sentences, tag, test_size=0.2) # делим на трейн и тест с помощью train_test_split

In [15]:
train_dataset = EntityDataset(
    texts=train_sentences, tags=train_tag
)

train_data_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=0, #
    shuffle=True, drop_last=True
)

valid_dataset = EntityDataset(
    texts=test_sentences, tags=test_tag
)

valid_data_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=VALID_BATCH_SIZE, num_workers=0, #
    shuffle=False, drop_last=True
)

In [16]:
def eval_model(model, valid_data_loader):
    h = model.init_hidden(VALID_BATCH_SIZE)
    losses = []
    
    correct_sum, total_sum = 0, 0
    
    for inputs, labels, mask in valid_data_loader:
        h = tuple([each.data for each in h])
        # отправим inputs, labels и mask на GPU
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
            mask = mask.cuda()
            
        model.zero_grad()
        output, h = model(inputs, h)
        loss = loss_fn(output, labels.flatten(), mask, num_tag)
        losses.append(loss.item())
        
        correct, total = acc_stat(torch.argmax(output, dim=-1).flatten(), labels.flatten(), mask.flatten())
        correct_sum += correct
        total_sum += total
    return losses, correct_sum / total_sum

In [17]:
hidden_dim = 512
n_layers = 2

model = EntityModel(num_tag, EMBED_DIM, hidden_dim, n_layers, drop_prob=0.5, bidirectional=False)
model.to(device)

lr=0.005
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [18]:
counter = 0
print_every = 10
clip = 5
valid_loss_min = np.Inf
writer = SummaryWriter('logs')


model.train()
for i in range(EPOCHS):
    h = model.init_hidden(TRAIN_BATCH_SIZE)
    
    correct_sum, total_sum = 0, 0
    
    for inputs, labels, mask in tqdm(train_data_loader): #
        counter += 1
        h = tuple([e.data for e in h])

        inputs = inputs.to(device)
        labels = labels.to(device)
        mask = mask.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = loss_fn(output, labels.flatten(), mask, num_tag) # вызываем функцию для подсчета лосса
        loss.backward() # и делаем обратное распространение ошибки
        correct, total = acc_stat(torch.argmax(output, dim=-1).flatten(), labels.flatten(), mask.flatten()) # вызываем функцию acc_stat
        correct_sum += correct
        total_sum += total

        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step() # градиентный спуск
        
        if counter % print_every == 0:
            model.eval()
            val_losses, val_acc = eval_model(model, valid_data_loader)
            model.train()
            
            val_loss = np.mean(val_losses)
            writer.add_scalar('train/loss', loss.item(), counter)
            writer.add_scalar('val/loss', val_loss, counter)
            writer.add_scalar('train/acc', correct_sum / total_sum, counter)
            writer.add_scalar('val/acc', val_acc, counter)

            print("Epoch: {}/{}...".format(i+1, EPOCHS),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(val_loss),
                  "Train Acc: {:.6f}".format(correct_sum / total_sum),
                  "Val Acc: {:.6f}".format(val_acc))
                
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), MODEL_PATH)
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)

num_directions 1


  3%|▎         | 9/299 [00:05<02:47,  1.74it/s]

num_directions 1


  3%|▎         | 10/299 [00:34<45:45,  9.50s/it]

Epoch: 1/5... Step: 10... Loss: 0.884185... Val Loss: 0.766111 Train Acc: 0.766869 Val Acc: 0.846548
Validation loss decreased (inf --> 0.766111).  Saving model ...


...

 98%|█████████▊| 293/299 [17:18<00:05,  1.06it/s]

num_directions 1


 98%|█████████▊| 294/299 [17:49<00:49,  9.99s/it]

Epoch: 5/5... Step: 1490... Loss: 0.112944... Val Loss: 0.130492 Train Acc: 0.957220 Val Acc: 0.959828
Validation loss decreased (0.130719 --> 0.130492).  Saving model ...


100%|██████████| 299/299 [17:52<00:00,  3.59s/it]


## Inference

In [20]:
meta_data = joblib.load("meta.bin")
enc_tag = meta_data["enc_tag"]

num_tag = len(list(enc_tag.classes_))

text = """
Natasha is traveling to New York
"""

device = torch.device("cuda")
model.to(device)

# так как это инференс, выключаем расчет градиентов:
with torch.no_grad():
    # inputs = torch.tensor([fasttext_model.wv[s] for s in word_tokenize(text)], dtype=torch.float32)
    inputs = torch.tensor([fasttext_model[s] for s in word_tokenize(text)], dtype=torch.float32)
    inputs = inputs.unsqueeze(0).to(device)
    h = model.init_hidden(1)
    tag, h = model(inputs, h)

    print(
        enc_tag.inverse_transform(
            tag.argmax(-1).cpu().numpy().reshape(-1)
        )
    )

C:\Temp\ipykernel_13004\1332789944.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  inputs = torch.tensor([fasttext_model[s] for s in word_tokenize(text)], dtype=torch.float32)


num_directions 1
['B-per' 'O' 'O' 'O' 'B-geo' 'I-geo']
